<a href="https://colab.research.google.com/github/abhi147p/Abhishek_INFO5731_Spring2023/blob/main/In_class_exercise_02_02072023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The second In-class-exercise (02/07/2023, 40 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points): Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:

Now a days, Machine Learning is rapidly rising towards future everywhere, where it needs huge amount of data for performing different algorithms by analysing
it. We generally use different process to collect and analyse data using ML algorithms for different topics such as regression analysis to categorise
the data into different parts and clustering the data etc. What if we want to know:
-> what all topics are trending over machine learning
-> what new packages have been deployed and packages that are deprecated.

Here the data, I want, will get through reddit website.

Research Questions:

-> What process is needed to acheive such thing?
-> How can we find such solution based on data? 
-> What algorithms models are needed for our analysis?
-> How does sentiment analysis works over our data, that we scrap through website.

The data that will be needed in this process is, user_id, title, description, number_of_upvotes and downvotes, so that we can get to know which all 
posts are on trending. For analysis of this process, we need a huge amount of data, around 3000-5000 records of data

The steps to collect data:
-> For this process, we use reddit site to scrape.
-> With the help of praw api, we will scrape the reddit website data and save it in csv format.

'''

Question 2 (10 points): Write python code to collect 1000 data samples you discussed above.

In [49]:
# You code here (Please add comments in the code):

import praw, time
import pandas as pd
import datetime as dt

def get_date(created):
  """
    params1: created_at time from praw
    return: converted into datetime format
  """
  return dt.datetime.fromtimestamp(created)

def authenticate_reddit():
  """
    return: praw api, that has been authenticated
  """
  reddit = praw.Reddit(
      client_id="yCq-n_3-AtNM-6V1z97-Gg",
      client_secret="ET7VSf-Pg-CuqJNQrvv0WhFm7oPtiA",
      password="***********",
      user_agent="testscript by u/Abhishek_Abhi147", #here for default we can use testscript
      username="Abhishek_Abhi147",
      check_for_async=False
  )
  return reddit

def fetch_reddit_data(reddit, count):
  """
    params1: praw api of reddit
    params2: count, number of posts
    return: dataframe, which contains N number of posts data that has been scraped
  """
  # using MachineLearning as topic and for the trending posts, we used top operator with number of posts as limit
  subreddit_posts = reddit.subreddit('MachineLearning').top(limit=count)
  data = []
  i = 0
  for post in subreddit_posts:
    data.append([post.id, post.title, post.selftext, post.created, post.score])
    i = i + 1
    if i%50 == 0:
      time.sleep(3)

  df = pd.DataFrame(data, columns=["Id", "Title", "Description", "created_at", "Likes or upvotes"])
  # converting to datetime format
  df['created_at'] = df['created_at'].apply(get_date)
  return df

reddit = authenticate_reddit()
print(f"user_name: {reddit.user.me()}")
posts_count = int(input("Enter number of posts: "))
df = fetch_reddit_data(reddit, posts_count)
print(f"The size of data: {df.shape[0]}")
df.head()

user_name: Abhishek_Abhi147
Enter number of posts: 1000
The size of data: 998


,Id,Title,Description,created_at,Likes or upvotes
0,gh1dj9,[Project] From books to presentations in 10s w...,,2020-05-10 13:19:54,7731
1,kuc6tz,[D] A Demo from 1993 of 32-year-old Yann LeCun...,,2021-01-10 10:30:36,5799
2,g7nfvb,[R] First Order Motion Model applied to animat...,,2020-04-25 04:27:23,4742
3,lui92h,[N] AI can turn old photos into moving Images ...,,2021-02-28 15:12:28,4665
4,ohxnts,[D] This AI reveals how much time politicians ...,,2021-07-11 04:18:59,4520


Question 3 (10 points): Write python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "information retrieval". The articles should be published in the last 10 years (2012-2022).

The following information of the article needs to be collected:

(1) Title

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [9]:
# You code here (Please add comments in the code):

import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd


def get_soup(url):
    """
        params1: url (contains the url of google scholar page)
        return: soup (fetching the url page data and then further converted to html parser)
    """
  # headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    try:
        # requesting for data using requests with url and headers for authentication
        data = requests.get(url, headers)
        # print(f"Extracted the data with response as {data.status_code}")
        if data.status_code != 200:
            raise Exception("Failed to fetch data")
    except Exception as ex:
        print(f"Exception occurred as {data.text} with status_code {data.status_code}")
        return None
    soup = BeautifulSoup(data.content)
    return soup

def get_title(title):
    """
      params1: soup_api with fetched title of article
      return: string format of text title of article
    """
    return str(title.find("a").text)

def get_abstract_url(title):
    """
      params1: soup_api with fetched title of article
      return: string format of url for title of article
    """
    return str(title.find("a").get("href"))


def get_article_info(article):
    """
      params1: soup_api with fetched article info
      return: tuples containing author, year, and published info 
    """
    # using regular expressions, fetched year from article
    year = int(re.search(r'\d+', article.text).group())
    # performing some string operations, fetched required results
    article = str(article.text).replace("\xa0", "")
    article = article.split("-")
    published = article[-1].strip()
    author = article[0].strip()
    return author, year, published

def get_tags(soup):
    """
      params1: soup_api with fetched url and parsed data
      return: list of article info, such as titles, authors, year, published, abstract
    """
    # fetched titles and authors of article using findAll by mentioning some tags
    all_titles = soup.findAll("h3", attrs={"class": "gs_rt"})
    all_authors = soup.findAll("div", attrs={"class": "gs_a"})
    all_abstracts = soup.findAll("div", attrs={"class": "gs_rs"})

    authors, year, published = [], [], []

    titles = [get_title(title) for title in all_titles]
    abs_url = [get_abstract_url(title) for title in all_titles]
    abstract = [get_abstract(abstr) for abstr in all_abstracts]
 
    for author in all_authors:
        auth, yr, publs = get_article_info(author)
        authors.append(auth)
        year.append(yr)
        published.append(publs)
 
    return titles, authors, year, published, abstract, abs_url
 

def get_abstract(abstr):
    """
      params1: soup_Api with fetched abstract of article
      return: string format of article abstract by fetching its text
    """
    return str(abstr.text).replace("\n", "")


def fetch_web_data(records):
    """
      params1(records): number of articles, needs to be fetched
      return: dataframe containing total N number of articles.
    """
    year_st, year_end = 2012, 2022
    columns_google = ["Title", "Author", "Year", "Published", "Abstract", "Abstract_UrL"]
    # fetching for 1000 articles
    titles, authors, years, published, abstract, abs_url = [], [], [], [], [], []
    final_data = []
    print("***** BEFORE FETCHING *********")
    # records = 100 # no of articles
    for i in range(0, records, 10):
        url = f"https://scholar.google.com/scholar?start={i}&q=information+retrieval&hl=en&as_sdt=0,44&as_ylo={year_st}&as_yhi={year_end}&as_vis=1"
        soup = get_soup(url)
        if soup is None:
          print(f"Data Not Fetched.... for {i} article page")
          continue
        # titles, authors, year, published, abstract, abs_url
        a, b, c, d, e, f = get_tags(soup)
        titles.extend(a)
        authors.extend(b)
        years.extend(c)
        published.extend(d)
        abstract.extend(e)
        abs_url.extend(f)
        print(f"******* fetched {(i+10)} articles *********")
        # keeping time to sleep for 5 seconds, so that, server may not crash for frequent multiple requests.
        time.sleep(5)
   
    for i in range(records):
        final_data.append([titles[i],authors[i],years[i],published[i],abstract[i],abs_url[i]])

    print("******* AFTER FETCHING ********")
    df = pd.DataFrame(final_data, columns = columns_google)
    print(f"Number of records: {df.shape[0]}")
    return df

# url = "https://scholar.google.com/scholar?start=200&q=information+retrieval&hl=en&as_sdt=0,44&as_ylo=2012&as_yhi=2022&as_vis=1"
# soup = get_soup(url)
# all_titles = soup.findAll("h3", attrs={"class": "gs_rt"})
# all_authors = soup.findAll("div", attrs={"class": "gs_a"})
# all_abstracts = soup.findAll("div", attrs={"class": "gs_rs"})

df = fetch_web_data(int(input("Enter num of articles: ")))
print(f"dimensions of articles: {df.shape}")
df.head()

Enter num of articles: 100
***** BEFORE FETCHING *********
******* fetched 10 articles *********
******* fetched 20 articles *********
******* fetched 30 articles *********
******* fetched 40 articles *********
******* fetched 50 articles *********
******* fetched 60 articles *********
******* fetched 70 articles *********
******* fetched 80 articles *********
******* fetched 90 articles *********
******* fetched 100 articles *********
******* AFTER FETCHING ********
Number of records: 100
dimensions of articles: (100, 6)


,Title,Author,Year,Published,Abstract,Abstract_UrL
0,Information retrieval as statistical translation,"A Berger, J Lafferty",2017,dl.acm.org,… There is a large literature on probabilistic...,https://dl.acm.org/doi/abs/10.1145/3130348.313...
1,A survey of automatic query expansion in infor...,"C Carpineto, G Romano",2012,dl.acm.org,… information retrieval systems is largely cau...,https://dl.acm.org/doi/abs/10.1145/2071389.207...
2,A language modeling approach to information re...,"JM Ponte, WB Croft",2017,dl.acm.org,"… models, we have developed an approach to ret...",https://dl.acm.org/doi/pdf/10.1145/3130348.313...
3,A study of smoothing methods for language mode...,"C Zhai, J Lafferty",2017,dl.acm.org,… to information retrieval are attractive and ...,https://dl.acm.org/doi/abs/10.1145/3130348.313...
4,Open language learning for information extraction,"M Schmitz, S Soderland, R Bart…",2012,aclanthology.org,Open Information Extraction (IE) systems extra...,https://aclanthology.org/D12-1048.pdf


Question 4 (10 points): Write python code to collect 1000 posts from Twitter, or Facebook, or Instagram. You can either use hashtags, keywords, user_name, user_id, or other information to collect the data. 

The following information needs to be collected:

(1) User_name

(2) Posted time

(3) Text 

In [2]:
# You code here (Please add comments in the code):

# hashtag

import tweepy
import pandas as pd

consumer_key = "u7L1lnR7HN85dn1qnTFO1cegb"
consumer_secret = "QN1JrEmit2To46ZcwWAT4aI5QGWZXWRDDUPnMCWV5M66SFc8wT"
access_key = "1144377060036620294-BSEicX3zH7hIhksbNZV9mrWFwa07cO"
access_secret = "gxWMOodDq1nQAjix9mHEOUSAtgE7XH5ctHInm0XRslJce"

def authenticate():
    # Using OAuthHandler to authorize into twitter account by passing consumer_key, consumer_secret, access_key and access_secret
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    # getting tweepy API by sending performed authentication
    api = tweepy.API(auth)
    return api

def fetch_tweets(api, num_of_tweets, hashtag):
    """
      params1: api, is an twitter api(tweepy), that has been authenticated to get the data from twitter
      params2: num_of_tweets, total amount of tweets or posts required
      params3: hashtag, topic or name on which we want to search for tweet
      return: list of N num of all_tweets containing each tweet with info.
    """
    # all_tweets = []
    """ 
    With the help of tweepy Cursor, using search_api and hashtag query as parameters, fetching all the tweets from twitter on particular
    topic of hashtag. The number of tweets to be fetched is handled with the help of items(num_of_tweets)
    """
    # for tweet in tweepy.Cursor(api.search, q='#' + hashtag).items(num_of_tweets):
    #     all_tweets.append(tweet)
    all_tweets = [tweet for tweet in tweepy.Cursor(api.search, q='#' + hashtag).items(num_of_tweets)]
    return all_tweets

def fetch_tweet_info(tweet):
    """
      params1: tweet, containing each post of tweet info
      return: list of user_name, tweet_text and tweet_posted_at
    """
    # fetching each tweet info, such as user_name, tweet_text, posted_time
    return [str(tweet.user.screen_name), str(tweet.text), str(tweet.created_at)]

def get_Tweets():
    """
      return: dataframe containing the list of all tweets info with username, tweet_text, posted_time
    """
    # Authentication of twitter devloper account
    tweet_api = authenticate()
    num_of_tweets = int(input("Enter number of tweets want to fetch: "))
    hashtag = input("Enter the hashtag: ")
    all_tweets = fetch_tweets(tweet_api, num_of_tweets, hashtag)
    # fetching each tweet info
    tweet_info = list(map(fetch_tweet_info, all_tweets))
    # setting all tweets info into dataframe with columns of usermname, text and posted_time, to analyze our data more effectively using numpy and pandas
    df = pd.DataFrame(tweet_info, columns=["username", "text", "posted_time"])
    print(f"dimensions of tweets: {df.shape}")
    return df

df = get_Tweets()
df.head()

Enter number of tweets want to fetch: 1000
Enter the hashtag: Sports
dimensions of tweets: (1000, 3)


,username,text,posted_time
0,MGuindonRC,Belleterre est le dernier endroit au Québec qu...,2023-02-12 21:31:08
1,TesoroMoctezuma,😆🧋🥪#FelizDomingo!#EnPortada #Portada de\n#Port...,2023-02-12 21:31:00
2,verde2730,RT @SportsHochi: #Tリーグ ・東京がファイナル進出に王手 世界5位・カルデ...,2023-02-12 21:30:20
3,verde2730,RT @SportsHochi: #Tリーグ ・日本生命、森さくらが殊勲の勝利 #平野美宇...,2023-02-12 21:30:14
4,hoodgrindvideo,...... - https://t.co/ymCbaDGDoQ\n#hoodgrind #...,2023-02-12 21:30:04
